In [1]:
import sys
from timeit import default_timer as timer
import pmm.inputs
import pmm.pmm 
import pmm.conversions
import MDAnalysis as mda
import numpy as np
#import numpy.linalg 
import nglview as nv

In [2]:
# gather the electronic properties
start = timer()
gauss_file = 'water/qm/wat_esp{}.log'
n_el_states = 4
qm_inputs = pmm.inputs.get_tot_input_gauss(gauss_file, n_el_states)
end = timer()
print('inputs da Gaussian ci mette: ', end - start)
#print(qm_inputs)

qc_geom = pmm.pmm.convert2Universe(qm_inputs['geometry'])
# shift origin to center of mass
cdm_ref = qc_geom.atoms.center_of_mass()
qc_geom.atoms.positions = qc_geom.atoms.positions - cdm_ref

print(qc_geom)


inputs da Gaussian ci mette:  0.09268159998464398
<Universe with 3 atoms>
/mnt/d/miniconda3/envs/pmm/lib/python3.9/site-packages/MDAnalysis/core/universe.py:467: UserWarning: Residues specified but no atom_resindex given.  All atoms will be placed in first Residue.
  warnings.warn(
/mnt/d/miniconda3/envs/pmm/lib/python3.9/site-packages/MDAnalysis/core/universe.py:473: UserWarning: Segments specified but no segment_resindex given.  All residues will be placed in first Segment
  warnings.warn(


In [3]:
# Read the MD trajectory
start = timer()
traj = mda.Universe('water/mm/md.tpr', 'water/mm/trajout.xtc')
end = timer()
print('leggere xtc ci mette: ', end - start)
w = nv.show_mdanalysis(traj)
w

#qc_traj, solv_traj = pmm.split_qc_solv(traj, '1:28')
#print(qc_traj)
#w = nv.show_mdanalysis(solv_traj)
#w

leggere xtc ci mette:  0.13068349999957718


NGLWidget(max_frame=10)

In [4]:
# MD-PMM calculation

start = timer()
el_field_traj = []
potential_traj = []
rot_dip_matrix_traj = []
eigenvals = []
eigenvecs = []
for ts in traj.trajectory:
    # Divide the trajectory in QC and solvent.
    # NOTE: the indexes for the QC are inclusive of the extremes.
    qc_traj, solv_traj = pmm.pmm.split_qc_solv(traj, '1:3')
    #print(pmm.rotate_dip_matrix(qm_inputs['dip_matrix'], qc_traj, qc_geom))

    # Calculate the center of mass of QC
    cdm = qc_traj.center_of_mass()
    print('cdm: ', cdm)
    #print(solv_traj.atoms.positions)

    # Calculate the el_field
    el_field, potential = pmm.pmm.calc_el_field_pot(solv_traj.atoms.positions, solv_traj.atoms.charges, cdm)
    el_field_traj.append(el_field)
    potential_traj.append(potential)
    #print(el_field, potential)

    # Rotate the dip_matrix using mwsfit
    rot_dip_matrix = pmm.pmm.rotate_dip_matrix(qm_inputs['dip_matrix'], qc_traj, qc_geom)
    rot_dip_matrix_traj.append(rot_dip_matrix)

    pmm_matrix = pmm.pmm.calc_pmm_matrix(qm_inputs['energies'], rot_dip_matrix, el_field, potential)
    #print(pmm_matrix)
    eigenval, eigenvec = np.linalg.eigh(pmm_matrix)
    eigenvals.append(eigenval)
    eigenvecs.append(eigenvecs)
end = timer()
print('PMM ci mette: ', end - start)



cdm:  [ 9.87342911 10.00740544 17.62475016]
cdm:  [ 6.69084518 11.9881194   1.09230951]
cdm:  [ 4.95895262 10.33707098  1.74853561]
cdm:  [ 9.57622659 17.75517984  4.22292904]
cdm:  [11.81650031  0.02859542  2.34691695]
cdm:  [14.12377428  1.06811892  1.15028565]
cdm:  [15.72657307  1.99202393  3.03707119]
cdm:  [17.43210774 16.91223968  7.49097652]
cdm:  [17.59671446  3.92684573  3.45530971]
cdm:  [2.07328569 2.39104776 1.27322595]
cdm:  [18.01460721  1.12021418 18.34936978]
PMM ci mette:  0.06989220000104979
/mnt/d/miniconda3/envs/pmm/lib/python3.9/site-packages/MDAnalysis/core/selection.py:728: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.zeros(len(grou

In [5]:
# Check the electric field:
for frame in el_field_traj:
    print(*frame)

0.014276775284243067 0.0001400488857403685 -0.007042059413858543
0.027576742057973343 0.01774748757966043 -0.00929501898413013
-0.004084725427671502 0.01862958920583513 0.022133216820143922
0.0008672033278938741 0.004544040082551991 -0.021743179290322708
-0.006938866027541683 -0.013406364417855617 -0.0027878785743164964
-0.004948386250894784 0.003079120214502901 0.00746943591768773
-0.012921738544510274 -0.005887627130070764 0.024404794648605035
0.018288453988338053 0.016298081211073547 -0.011650802537297607
-0.003996725108249908 -0.025334598319905345 -0.009379358351242411
0.008433818943369344 -0.003519565112290671 0.011692594268626915
-0.0005489663945449422 0.018189412302576626 -0.007512655224413784


In [6]:
# Check potential
for frame in potential_traj:
    print(frame)

-0.0007186269082465821
-0.06924384467727693
0.045158613158056984
0.050415757928526254
0.029345873629465793
-0.02370834123182902
0.016911266739411124
0.046111959721585355
0.05410787887327574
-0.03738909116874445
-0.0671659422772331


In [7]:
# Check the rotated dip matrix
for frame in rot_dip_matrix_traj:
    print(frame)

[[[ 0.17087604 -0.66996144 -0.44885683]
  [-0.16682237  0.08317261 -0.18765091]
  [ 0.          0.          0.        ]
  [-0.12572193  0.49292368  0.33024611]]

 [[-0.16682237  0.08317261 -0.18765091]
  [-0.04149765  0.16270172  0.10900594]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 [[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [-0.02054493  0.08055142  0.05396737]
  [ 0.          0.          0.        ]]

 [[-0.12572193  0.49292368  0.33024611]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [-0.04280261  0.16781813  0.1124338 ]]]
[[[ 0.51724218  0.6220266   0.15831246]
  [ 0.01951933  0.0497009  -0.25905418]
  [ 0.          0.          0.        ]
  [-0.38056059 -0.45765566 -0.11647829]]

 [[ 0.01951933  0.0497009  -0.25905418]
  [-0.12561349 -0.15106063 -0.03844656]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 [[ 0.          0.          